## Objective
1. Visualize the launches in an informative way
2. Identify any outlier launches that should be reviewed
3. Identify any interesting patterns in the data (seasonality, poorly performing parts)

## 1. Visualize the launches in an informative way
- The data contains only 5 seconds prior to 15 seconds after launch
- I assume the presuption is that something strange might be happening after the zip is launched
- Need to detect outlier launches, which would mean that the distance or speed after the launch is not as high as it could be when one accounts for the wind speed

### a. load the data into a single dataframe

In [1]:
import pandas as pd
import re
from glob import glob
import yaml

from sklearn.decomposition import PCA
import plotly.plotly as py
import plotly.graph_objs as go

In [2]:
with open('creds.yaml', 'r') as f:
    creds = yaml.load(f)

In [3]:
import plotly
plotly.tools.set_credentials_file(username=creds['plotly']['username'], 
                                  api_key=creds['plotly']['apikey'])

In [4]:
hires_flight_csv = glob('../data/flight*.csv')

In [5]:
reg = '../data/flight_(\d*)'

In [6]:
hires_flight_data = pd.DataFrame()
for csv in hires_flight_csv:
    csv_data = pd.read_csv(csv)
    flight_number = re.match(reg, csv).group(1)
    csv_data['flight_id'] = int(flight_number)
    hires_flight_data = pd.concat([hires_flight_data, csv_data])

### b. Check that the range of values for the time after launch is the same
- This is important because if the distance traveled is the metric, then the time after the launch needs to be the same

In [7]:
hires_flight_data.columns

Index(['seconds_since_launch', 'position_ned_m[0]', 'position_ned_m[1]',
       'position_ned_m[2]', 'velocity_ned_mps[0]', 'velocity_ned_mps[1]',
       'velocity_ned_mps[2]', 'accel_body_mps2[0]', 'accel_body_mps2[1]',
       'accel_body_mps2[2]', 'orientation_rad[0]', 'orientation_rad[1]',
       'orientation_rad[2]', 'angular_rate_body_radps[0]',
       'angular_rate_body_radps[1]', 'angular_rate_body_radps[2]',
       'position_sigma_ned_m[0]', 'position_sigma_ned_m[1]',
       'position_sigma_ned_m[2]', 'flight_id'],
      dtype='object')

In [8]:
max_time_after_launch = hires_flight_data.groupby('flight_id').\
                                          agg({'seconds_since_launch':'max'})

In [9]:
max_time_after_launch.describe()

,seconds_since_launch
count,447.000000
mean,14.995431
std,0.000100
min,14.995020
25%,14.995380
50%,14.995420
75%,14.995470
max,14.996050


It looks like almost all of the data has a datapoint within one one hunderedth of the 15 second mark. This will make it easy to compare one flight with another.

One metric that could be important is the velocity after 15 seconds. This metric would be important if every flight was relatively straight. Plot the positions for a random set of flights 

### c. Plot the path of a sample of planes

In [10]:
flight_summaries = pd.read_csv('../data/summary_data.csv')

In [11]:
flight_sample = flight_summaries.sample(n=10, random_state=42)

In [12]:
hires_flight_data.columns

Index(['seconds_since_launch', 'position_ned_m[0]', 'position_ned_m[1]',
       'position_ned_m[2]', 'velocity_ned_mps[0]', 'velocity_ned_mps[1]',
       'velocity_ned_mps[2]', 'accel_body_mps2[0]', 'accel_body_mps2[1]',
       'accel_body_mps2[2]', 'orientation_rad[0]', 'orientation_rad[1]',
       'orientation_rad[2]', 'angular_rate_body_radps[0]',
       'angular_rate_body_radps[1]', 'angular_rate_body_radps[2]',
       'position_sigma_ned_m[0]', 'position_sigma_ned_m[1]',
       'position_sigma_ned_m[2]', 'flight_id'],
      dtype='object')

In [13]:
set(flight_summaries.columns).intersection(set(hires_flight_data.columns))

{'flight_id'}

In [14]:
select_flights = flight_sample[['flight_id']].merge(hires_flight_data, 
                                    how='inner',
                                    on='flight_id')
select_flights.reset_index
select_flights.head(2)

,flight_id,seconds_since_launch,position_ned_m[0],position_ned_m[1],position_ned_m[2],velocity_ned_mps[0],velocity_ned_mps[1],velocity_ned_mps[2],accel_body_mps2[0],accel_body_mps2[1],accel_body_mps2[2],orientation_rad[0],orientation_rad[1],orientation_rad[2],angular_rate_body_radps[0],angular_rate_body_radps[1],angular_rate_body_radps[2],position_sigma_ned_m[0],position_sigma_ned_m[1],position_sigma_ned_m[2]
0,17508,-4.99813,4.511611,7.207816,-3.410138,0.0,0.0,0.0,2.224452,0.016884,-9.503209,0.006961,0.216883,2.740921,-0.000893,0.000388,0.002370,0.144418,0.168294,0.394049
1,17508,-4.97848,4.508890,7.205891,-3.415132,0.0,0.0,0.0,2.177743,-0.018896,-9.506433,0.006963,0.216905,2.740921,0.002406,-0.004695,0.000523,0.144420,0.168297,0.394055


In [15]:
coordinates = ['flight_id', 'seconds_since_launch', 'position_ned_m[0]', 'position_ned_m[1]',
       'position_ned_m[2]']

In [16]:
select_flight_groups = select_flights.groupby('flight_id')

In [17]:
traces = []
for flight, flight_data in select_flight_groups:
    temp_trace = go.Scatter3d(x=flight_data['position_ned_m[1]'],
                            y=flight_data['position_ned_m[0]'],
                            z=flight_data['position_ned_m[2]'] * (-1),
                            hovertext = flight_data['seconds_since_launch'],
                            mode='lines',
                            name=flight)
    traces.append(temp_trace)
layout = go.Layout(title='Trajectory for 10 random flights')
fig = go.Figure(data=traces, layout=layout)
py.iplot(fig, filename='trajectory')

/home/obdsd/Documents/workspace/zipline_tsq/dsth_zip/v36/lib/python3.6/site-packages/IPython/core/display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



In [18]:
traces = []
for flight, flight_data in select_flight_groups:
    temp_trace = go.Scatter3d(x=flight_data['velocity_ned_mps[1]'],
                            y=flight_data['velocity_ned_mps[0]'],
                            z=flight_data['velocity_ned_mps[2]'],
                            hovertext = flight_data['seconds_since_launch'],
                            mode='lines+markers',
                            name=flight)
    traces.append(temp_trace)
layout = go.Layout(title='Velocity for 2 random flights')
fig = go.Figure(layout=layout, data=traces)
py.iplot(fig, filename='velocity')

It looks like the flights are primarly along the same XY plane

## 2. Quantify the deviation from the flight path
First make sure that the number of datapoints that we get from each flight is the same

In [19]:
flight_data_points = hires_flight_data.groupby('flight_id').\
                                          agg({'seconds_since_launch':'count'})
flight_data_points.describe()

,seconds_since_launch
count,447.000000
mean,1000.617450
std,3.132208
min,945.000000
25%,1001.000000
50%,1001.000000
75%,1001.000000
max,1001.000000


In [20]:
flights_with_good_data = flight_data_points.loc[flight_data_points.seconds_since_launch == 1001]
flights_with_good_data.shape

(427, 1)

In [21]:
position_columns = ['seconds_since_launch', 'position_ned_m[0]', 'position_ned_m[1]','position_ned_m[2]',
                    'orientation_rad[0]', 'orientation_rad[1]', 'orientation_rad[2]']

In [22]:
downsampled_data = hires_flight_data.copy()
downsampled_data['seconds_since_launch'] = downsampled_data.seconds_since_launch.round(1)
flight_maxmin = downsampled_data[position_columns].groupby('seconds_since_launch').agg(['mean', 'std'])
cols = pd.Series(flight_maxmin.columns.tolist()).apply(pd.Series).sum(axis=1)
flight_maxmin.columns = cols
flight_maxmin.head()

,position_ned_m[0]mean,position_ned_m[0]std,position_ned_m[1]mean,position_ned_m[1]std,position_ned_m[2]mean,position_ned_m[2]std,orientation_rad[0]mean,orientation_rad[0]std,orientation_rad[1]mean,orientation_rad[1]std,orientation_rad[2]mean,orientation_rad[2]std
seconds_since_launch,,,,,,,,,,,,
-5.0,4.595146,0.695278,7.152763,0.914166,-4.505521,1.776276,0.007222,0.001086,0.217737,0.001777,2.714132,0.282433
-4.9,4.594823,0.695072,7.152820,0.914134,-4.505010,1.776009,0.007223,0.001087,0.217735,0.001777,2.714112,0.282496
-4.8,4.594818,0.694540,7.153023,0.913831,-4.504880,1.775948,0.007222,0.001086,0.217736,0.001778,2.714112,0.282496
-4.7,4.595287,0.693836,7.153229,0.913008,-4.505157,1.774357,0.007221,0.001085,0.217737,0.001777,2.714148,0.282307
-4.6,4.595382,0.693116,7.153009,0.912356,-4.505396,1.773566,0.007222,0.001085,0.217737,0.001777,2.714172,0.282182


In [23]:
data_and_agg = flight_maxmin.merge(downsampled_data[['flight_id', 'seconds_since_launch', 'position_ned_m[0]', 'position_ned_m[1]','position_ned_m[2]',
                    'orientation_rad[0]', 'orientation_rad[1]', 'orientation_rad[2]']], 
                                   left_index=True, 
                                   right_on='seconds_since_launch')    
data_and_agg.head()

,position_ned_m[0]mean,position_ned_m[0]std,position_ned_m[1]mean,position_ned_m[1]std,position_ned_m[2]mean,position_ned_m[2]std,orientation_rad[0]mean,orientation_rad[0]std,orientation_rad[1]mean,orientation_rad[1]std,orientation_rad[2]mean,orientation_rad[2]std,flight_id,seconds_since_launch,position_ned_m[0],position_ned_m[1],position_ned_m[2],orientation_rad[0],orientation_rad[1],orientation_rad[2]
0,4.595146,0.695278,7.152763,0.914166,-4.505521,1.776276,0.007222,0.001086,0.217737,0.001777,2.714132,0.282433,16995,-5.0,4.335297,6.089418,-3.268693,0.005853,0.215869,2.740676
1,4.595146,0.695278,7.152763,0.914166,-4.505521,1.776276,0.007222,0.001086,0.217737,0.001777,2.714132,0.282433,16995,-5.0,4.335454,6.089682,-3.267985,0.005857,0.215868,2.740677
2,4.595146,0.695278,7.152763,0.914166,-4.505521,1.776276,0.007222,0.001086,0.217737,0.001777,2.714132,0.282433,16995,-5.0,4.335454,6.089682,-3.267985,0.005868,0.215866,2.740679
0,4.595146,0.695278,7.152763,0.914166,-4.505521,1.776276,0.007222,0.001086,0.217737,0.001777,2.714132,0.282433,16996,-5.0,4.419536,7.101200,-3.966274,0.006896,0.220499,2.740918
1,4.595146,0.695278,7.152763,0.914166,-4.505521,1.776276,0.007222,0.001086,0.217737,0.001777,2.714132,0.282433,16996,-5.0,4.418713,7.098773,-3.965782,0.006894,0.220513,2.740918


In [24]:
for column in ['position_ned_m[0]', 'position_ned_m[1]','position_ned_m[2]',
                'orientation_rad[0]', 'orientation_rad[1]', 'orientation_rad[2]']:
    mean_values = data_and_agg[column + 'mean']
    std_values = data_and_agg[column + 'std']
    data_and_agg[column + '_scaled'] = (data_and_agg[column] - mean_values)/ (std_values)
    

In [25]:
data_and_agg.head(2)

,position_ned_m[0]mean,position_ned_m[0]std,position_ned_m[1]mean,position_ned_m[1]std,position_ned_m[2]mean,position_ned_m[2]std,orientation_rad[0]mean,orientation_rad[0]std,orientation_rad[1]mean,orientation_rad[1]std,...,position_ned_m[2],orientation_rad[0],orientation_rad[1],orientation_rad[2],position_ned_m[0]_scaled,position_ned_m[1]_scaled,position_ned_m[2]_scaled,orientation_rad[0]_scaled,orientation_rad[1]_scaled,orientation_rad[2]_scaled
0,4.595146,0.695278,7.152763,0.914166,-4.505521,1.776276,0.007222,0.001086,0.217737,0.001777,...,-3.268693,0.005853,0.215869,2.740676,-0.373734,-1.163186,0.696304,-1.260643,-1.051377,0.093985
1,4.595146,0.695278,7.152763,0.914166,-4.505521,1.776276,0.007222,0.001086,0.217737,0.001777,...,-3.267985,0.005857,0.215868,2.740677,-0.373509,-1.162898,0.696702,-1.256795,-1.051636,0.093987


In [26]:
data_and_agg.loc[(data_and_agg.flight_id == 17459) & (data_and_agg.seconds_since_launch == -5.0)]

,position_ned_m[0]mean,position_ned_m[0]std,position_ned_m[1]mean,position_ned_m[1]std,position_ned_m[2]mean,position_ned_m[2]std,orientation_rad[0]mean,orientation_rad[0]std,orientation_rad[1]mean,orientation_rad[1]std,...,position_ned_m[2],orientation_rad[0],orientation_rad[1],orientation_rad[2],position_ned_m[0]_scaled,position_ned_m[1]_scaled,position_ned_m[2]_scaled,orientation_rad[0]_scaled,orientation_rad[1]_scaled,orientation_rad[2]_scaled
0,4.595146,0.695278,7.152763,0.914166,-4.505521,1.776276,0.007222,0.001086,0.217737,0.001777,...,-10.847755,0.008737,0.219457,2.741319,-1.158167,1.624317,-3.570523,1.395240,0.968426,0.096258
1,4.595146,0.695278,7.152763,0.914166,-4.505521,1.776276,0.007222,0.001086,0.217737,0.001777,...,-10.847402,0.008730,0.219488,2.741317,-1.158372,1.623672,-3.570325,1.388510,0.985588,0.096253
2,4.595146,0.695278,7.152763,0.914166,-4.505521,1.776276,0.007222,0.001086,0.217737,0.001777,...,-10.847402,0.008723,0.219464,2.741315,-1.158372,1.623672,-3.570325,1.381904,0.972619,0.096247


## 3. Visualize the flight metrics with PCA

In [27]:
scaled_fleet_data = data_and_agg[['flight_id',
                                  'position_ned_m[0]_scaled', 
                                 'position_ned_m[1]_scaled', 
                                 'position_ned_m[2]_scaled', 
                                 'orientation_rad[0]_scaled', 
                                 'orientation_rad[1]_scaled', 
                                 'orientation_rad[2]_scaled']]

In [28]:
flight_metrics = scaled_fleet_data.groupby('flight_id').agg(['min', 'max'])

In [29]:
pca = PCA(n_components=1, random_state=42)

In [30]:
flight_metrics.head(2)

position_ned_m[0]_scaled           position_ned_m[1]_scaled  \
                               min       max                      min   
flight_id                                                               
16951                    -0.923789  0.821177                -0.376436   
16952                    -0.473083  0.873152                -0.130935   

                   position_ned_m[2]_scaled            \
               max                      min       max   
flight_id                                               
16951      1.87768                -1.202257  0.114939   
16952      1.60857                -0.863851  0.534604   

          orientation_rad[0]_scaled           orientation_rad[1]_scaled  \
                                min       max                       min   
flight_id                                                                 
16951                     -2.158473  2.251129                 -4.596042   
16952                     -1.923303  2.132618                 -2.339243   

                    orientation_rad[2]_scaled            
                max                       min       max  
flight_id                                                
16951      0.920131                 -0.079459  0.268467  
16952      1.312672                  0.086127  0.490588

In [31]:
flight_metrics_reduced = pd.DataFrame(index=flight_metrics.index,
                                      data=pca.fit_transform(flight_metrics),
                                      columns=['x1'])
                                  
flight_metrics_reduced.head(2)

,x1
flight_id,
16951,-0.214492
16952,-0.565775


In [32]:
pd.Series(index=flight_metrics.columns,data=pca.components_[0])

position_ned_m[0]_scaled   min    0.236066
                           max    0.478396
position_ned_m[1]_scaled   min   -0.465154
                           max   -0.080024
position_ned_m[2]_scaled   min   -0.015084
                           max    0.014396
orientation_rad[0]_scaled  min   -0.189283
                           max    0.241203
orientation_rad[1]_scaled  min   -0.062762
                           max   -0.011768
orientation_rad[2]_scaled  min   -0.454368
                           max   -0.433207
dtype: float64

In [33]:
pca.explained_variance_ratio_

array([0.40928993])

In [34]:
trace = go.Histogram(x=flight_metrics_reduced.x1)
layout = go.Layout(title='x1',
                   font=dict(size=22),
                  xaxis=dict(
        title='X1',
    ),
    yaxis=dict(
        title='Number of flights',
    ))
fig = go.Figure(data=[trace], layout=layout)
py.iplot(fig, filename='pca')

/home/obdsd/Documents/workspace/zipline_tsq/dsth_zip/v36/lib/python3.6/site-packages/IPython/core/display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



## Validation

In [35]:
strange_flights = flight_metrics_reduced.loc[flight_metrics_reduced.x1 > 15]
strange_flights

,x1
flight_id,
17136,21.165091
17437,20.925103
17438,20.562772
17439,20.964209


In [36]:
traces = []
for flight_id, x1 in strange_flights.iterrows():
    flight_data = hires_flight_data.loc[hires_flight_data.flight_id == flight_id]
    temp_trace = go.Scatter3d(x=flight_data['position_ned_m[1]'],
                            y=flight_data['position_ned_m[0]'],
                            z=flight_data['position_ned_m[2]'] * (-1),
                            hovertext = flight_data['seconds_since_launch'],
                            mode='lines',
                            name=flight_id)
    traces.append(temp_trace)
layout = go.Layout(title='Trajectory for bad flights')
fig = go.Figure(data=traces, layout=layout)
py.iplot(fig, filename='trajectory_bad_flights')

## What caused the bad flights

In [37]:
strange_flights_with_info = strange_flights.merge(flight_summaries, on='flight_id', how='inner')
strange_flights_with_info

,flight_id,x1,air_temperature,battery_serial_number,body_serial_number,commit,launch_airspeed,launch_groundspeed,launch_timestamp,preflight_voltage,rel_humidity,static_pressure,wind_direction,wind_magnitude,wing_serial_number
0,17136,21.165091,18.75,15SPJJJ10027028,577350132840894487,5c504d9a16,31.740801,30.221628,2018-09-11 18:54:21 CAT,32.048248,63.65,80567.424655,129.655238,1.879699,15SPJJJ09043062
1,17437,20.925103,20.80,15SPJJJ10048030,577350132840857611,5c504d9a16,34.289555,30.468917,2018-09-24 19:25:52 CAT,32.188694,57.00,80543.443320,166.383315,2.801589,15SPJJJ09024061
2,17438,20.562772,20.90,15SPJJJ10022048,577350132807348254,5c504d9a16,33.892624,29.928555,2018-09-24 19:30:15 CAT,31.989910,55.20,80602.023104,174.511100,2.701217,15SPJJJ09040032
3,17439,20.964209,21.10,15SPJJJ10023027,577348835962105883,5c504d9a16,33.233994,30.141856,2018-09-24 19:34:41 CAT,32.003956,56.10,80487.580779,179.700111,2.755526,15SPJJJ09019061


In [38]:
flight_summaries.wind_magnitude.describe()

count    447.000000
mean       2.359469
std        0.996348
min        0.188798
25%        1.703303
50%        2.307687
75%        3.006968
max        7.466193
Name: wind_magnitude, dtype: float64

In [39]:
flight_summaries.launch_airspeed.describe()

count    447.000000
mean      31.976493
std        1.759982
min       28.027149
25%       30.761058
50%       31.893215
75%       33.198513
max       36.929199
Name: launch_airspeed, dtype: float64

In [40]:
strange_flights_with_info[['flight_id', 'wind_magnitude', 'launch_groundspeed', 'launch_timestamp', 'commit']]

,flight_id,wind_magnitude,launch_groundspeed,launch_timestamp,commit
0,17136,1.879699,30.221628,2018-09-11 18:54:21 CAT,5c504d9a16
1,17437,2.801589,30.468917,2018-09-24 19:25:52 CAT,5c504d9a16
2,17438,2.701217,29.928555,2018-09-24 19:30:15 CAT,5c504d9a16
3,17439,2.755526,30.141856,2018-09-24 19:34:41 CAT,5c504d9a16


### Seasonality

In [41]:
summary_x1 = flight_summaries.merge(flight_metrics_reduced, left_on='flight_id', right_index=True)

In [42]:
date_trace = go.Scatter(x=summary_x1.launch_timestamp,
                          y=summary_x1.x1)
layout = go.Layout(title='Flight strangeness vs. Date',
                  font=dict(size=12))
fig = go.Figure(data=[date_trace], layout=layout)
py.iplot(fig, layout=layout, filename='seasonality')

## X1 by commit

In [43]:
summary_x1.commit.unique()

array(['5c504d9a16', '4d9468bd3c', '38bf99b15a', '1ecbc27833'],
      dtype=object)

In [44]:
commit_groups = summary_x1.groupby('commit')
traces = []
for commit, data in commit_groups:
    temp_trace = go.Histogram(x=data['x1'],
                        name=commit,
                              histnorm='percent',
                        opacity=0.5)
    traces.append(temp_trace)
layout = go.Layout(title='X1 vs commit',
                   yaxis=dict(title='x1'),
                   xaxis=dict(title='percent of flights'),
                   font=dict(size=22),
                   barmode='overlay')
fig = go.Figure(data=traces, layout=layout)
py.iplot(fig, filename='commit_group')

In [45]:
pca.components_

array([[ 0.23606646,  0.47839606, -0.46515372, -0.08002398, -0.01508359,
         0.01439605, -0.18928322,  0.24120295, -0.06276212, -0.01176827,
        -0.45436781, -0.43320731]])